In [13]:
"""
This script implements volatility models on clean data and produces estimates
"""

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from arch import arch_model
import math

In [14]:
def loadData(ticker):
    '''
    Load clean data and produce return series for analysis

    '''
    df = pd.read_csv("../clean/"+ticker+"_data.csv")
    Ret = df['Return']
    Ret = Ret[1:]
    return Ret

In [15]:
def evaluate(observation, forecast): 
    # Call sklearn function to calculate MAE
    mae = mean_absolute_error(observation, forecast)
    #print(f'Mean Absolute Error (MAE): {round(mae,3)}')
    # Call sklearn function to calculate MSE
    mse = mean_squared_error(observation, forecast)
    #print(f'Mean Squared Error (MSE): {round(mse,3)}')
    return math.sqrt(mse)

In [16]:
def modelGarch(ticker):
    '''
    Produce conditional volatility estimates from the three GARCH models.
    '''
    # Use loadData to produce the return list from the clean csv
    Ret = loadData(ticker)
    # Apply the GARCH specification
    skewt = arch_model(Ret, p=1, q=1,mean="constant",vol="GARCH",dist="skewt")    
    GJR = arch_model(Ret, p=1, q=1, o=1, mean="constant",vol="GARCH",dist="skewt")    
    E = arch_model(Ret, p=1, q=1, o=1, mean="constant",vol="EGARCH",dist="skewt")    
    # Fit the model
    skewtResult = skewt.fit(disp="off")
    GJRResult = GJR.fit(disp="off")
    EResult = E.fit(disp="off")
    return {"Ret": Ret, "SkewT": skewtResult, "GJR": GJRResult, "E": EResult}

def writeGarchVolatility(ticker):
    '''
    Processes conditional volatility estimates into csv files
    '''
    Ret = modelGarch(ticker)['Ret']
    skewtResult = modelGarch(ticker)['SkewT']
    GJRResult = modelGarch(ticker)['GJR']
    EResult = modelGarch(ticker)['E']
        # Take the conditional volatility from the model fit
    skewtVolatility = skewtResult.conditional_volatility
    GJRVolatility = GJRResult.conditional_volatility
    EVolatility = EResult.conditional_volatility

    garchVolatilityData = pd.concat([Ret, skewtVolatility, GJRVolatility, EVolatility], axis=1, names=['Return', 'SkewT', 'GJR', 'E'])
    garchVolatilityData.columns = ['Return', 'SkewT', 'GJR', 'E']
    garchVolatilityData.to_csv("../estimates/GARCHCondVol_" + ticker + ".csv")

# writeGarchVolatility("SPY")
# writeGarchVolatility("BTC-USD")

def resultGarch(ticker):
    '''
    Processes conditional volatility estimates into csv files
    '''
    Ret = modelGarch(ticker)['Ret']
    skewtResult = modelGarch(ticker)['SkewT']
    GJRResult = modelGarch(ticker)['GJR']
    EResult = modelGarch(ticker)['E']
        # Take the conditional volatility from the model fit
    skewtVolatility = skewtResult.conditional_volatility
    GJRVolatility = GJRResult.conditional_volatility
    EVolatility = EResult.conditional_volatility
    print(f'GARCH BIC: {skewtResult.bic}')
    print(f'GJR-GARCH BIC: {GJRResult.bic}')
    print(f'EGARCH BIC: {EResult.bic}')
    print (round(evaluate(Ret.sub(Ret.mean()).pow(2), skewtVolatility**2), 3))
    print (round(evaluate(Ret.sub(Ret.mean()).pow(2), GJRVolatility**2), 3))
    print (round(evaluate(Ret.sub(Ret.mean()).pow(2), EVolatility**2), 3))
    return {"GARCH_BIC": skewtResult.bic, 
        "GJR-GARCH BIC": GJRResult.bic,
        "EGARCH BIC": EResult.bic,
        

resultGarch("SPY")



GARCH BIC: 15263.707108947403
GJR-GARCH BIC: 15042.02801735152
EGARCH BIC: 15000.8844927105
5.097
4.92
4.876
